# Процессы

In [1]:
import random
import os
import time
import threading
import multiprocessing
import os

In [9]:
# Типы задач
def cpu_bound_task(n):
    def fib(n):
        if n == 0:
            return 0
        elif n == 1:
            return 1
        return fib(n-1) + fib(n-2)

    print(f'{n} -> {fib(n)}')


def io_bound_task(ind):
    print(f'start task {ind}')
    time.sleep(3)
    print(f'finish task {ind}\n', end='')

In [5]:
%%time

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    cpu_bound_task(number)

36 -> 14930352
35 -> 9227465
34 -> 5702887
33 -> 3524578
32 -> 2178309
31 -> 1346269
30 -> 832040
25 -> 75025
20 -> 6765
15 -> 610
10 -> 55
CPU times: user 14.4 s, sys: 40 ms, total: 14.5 s
Wall time: 14.5 s


In [6]:
%%time

# CPU bound задача на потоках
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = threading.Thread(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

20 -> 6765
25 -> 75025
15 -> 610
10 -> 55
30 -> 832040
31 -> 1346269
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
CPU times: user 14.5 s, sys: 146 ms, total: 14.7 s
Wall time: 14.6 s


In [7]:
%%time

# CPU bound задача на процессах
tasks = []

for number in [36, 35, 34, 33, 32, 31, 30, 25, 20, 15, 10]:
    task = multiprocessing.Process(target=cpu_bound_task, args=(number,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()


20 -> 6765
15 -> 610
10 -> 55
25 -> 75025
30 -> 832040
31 -> 1346269
32 -> 2178309
33 -> 3524578
34 -> 5702887
35 -> 9227465
36 -> 14930352
CPU times: user 22.3 ms, sys: 36.4 ms, total: 58.7 ms
Wall time: 6.19 s


In [10]:
%%time

for ind in range(5):
    io_bound_task(ind)

start task 0
finish task 0
start task 1
finish task 1
start task 2
finish task 2
start task 3
finish task 3
start task 4
finish task 4
CPU times: user 8.68 ms, sys: 3.46 ms, total: 12.1 ms
Wall time: 15 s


In [11]:
%%time

# IO bound задача на потоках
tasks = []

for ind in range(5):
    task = threading.Thread(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 0start task 1

start task 2start task 3
start task 4

finish task 1
finish task 3
finish task 4
finish task 2
finish task 0
CPU times: user 5.46 ms, sys: 4.61 ms, total: 10.1 ms
Wall time: 3.01 s


In [13]:
%%time

# IO bound задача на процессах
tasks = []

for ind in range(5):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 0
start task 1
start task 2
start task 3
start task 4
finish task 0
finish task 1
finish task 2
finish task 3
finish task 4
CPU times: user 15.6 ms, sys: 25.1 ms, total: 40.8 ms
Wall time: 3.04 s


In [15]:
# Получение идентификатора процесса
def io_bound_task(ind):
    print(f'start task {ind} with {os.getpid()}')
    time.sleep(3)
    print(f'finish task {ind}')
    
tasks = []

for ind in range(10):
    task = multiprocessing.Process(target=io_bound_task, args=(ind,))
    tasks.append(task)
    task.start()

for task in tasks:
    task.join()

start task 0 with 3625
start task 1 with 3626
start task 2 with 3627
start task 3 with 3628
start task 4 with 3629
start task 5 with 3630
start task 6 with 3631
start task 7 with 3632
start task 8 with 3633
start task 9 with 3634
finish task 0
finish task 1
finish task 2
finish task 3
finish task 4
finish task 5
finish task 6
finish task 7
finish task 8
finish task 9


In [16]:
# Пулл процессов
from multiprocessing import Pool


def getpid(n):
    time.sleep(2)
    return os.getpid()

with Pool(3) as p:
    print(p.map(getpid, range(5)))

[3637, 3638, 3639, 3638, 3639]


In [17]:
# Шаринг ресурсов
share_memory = {
    'count': 0,
}
def share_memory_task():
    print(f'read {share_memory["count"]}')
    share_memory['count'] += 1
    print(f'write {share_memory["count"]}')

tasks = []

for _ in range(5):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

read 0
write 1
read 0
write 1
read 0
write 1
read 0
write 1
read 0
write 1


In [18]:
# Шаринг ресурсов через файл
filename = 'share_memory.tmp'

with open(filename, 'w') as fd:
    fd.write('0')

def share_memory_task():
    with open(filename) as fd:
        count = int(fd.read())
    print(f'read {count}')

    with open(filename, 'w') as fd:
        fd.write(str(count + 1))
    print(f'write {count + 1}')


tasks = []

for _ in range(10):
    task = multiprocessing.Process(target=share_memory_task)
    tasks.append(task)
    task.start()
    
for task in tasks:
    task.join()

with open(filename) as fd:
    print(f'COUNT: {fd.read()}')

read 0
write 1
read 0
write 1
read 1
write 2
read 2
write 3
read 3
write 4
read 4
write 5
read 5
write 6
read 5
read 6
write 6
write 7
read 7
write 8
COUNT: 8


In [27]:
from multiprocessing import Manager, Queue


PROCESSES_COUNT = 5
to_process_list = [
    [1, 2, 3], [4, 5, 6], [7, 8, 9],
    [10, 11, 12], [13, 14, 15],
    [16, 17, 18], [19, 20, 21],
    [22, 23, 24], [25, 26, 27]
]


def func(x):
    time.sleep(3)
    return 1 << x


def producer_func(to_process, queue):
    for elem in to_process:
        res = func(elem)
        queue.put(res)
        print(f"Hey, I've just put {res} to queue!")


def consumer_func(queue):
    while True:
        res = queue.get()
        if res == 'kill':
            break
        print(f"Wow, I found {res} in queue!")


def do_work(to_process_list):
    manager = Manager()
    queue = manager.Queue()
    pool = Pool(PROCESSES_COUNT)

    pool.apply_async(consumer_func, (queue, ))

    jobs = []
    for to_process in to_process_list:
        job = pool.apply_async(producer_func, (to_process, queue))
        jobs.append(job)

    for job in jobs:
        job.get()

    queue.put('kill')
    pool.close()
    pool.join()
    

do_work(to_process_list)

Hey, I've just put 16 to queue!
Hey, I've just put 2 to queue!
Hey, I've just put 128 to queue!
Wow, I found 2 in queue!
Wow, I found 16 in queue!
Hey, I've just put 1024 to queue!
Wow, I found 128 in queue!
Wow, I found 1024 in queue!
Hey, I've just put 4 to queue!
Hey, I've just put 32 to queue!
Hey, I've just put 256 to queue!
Wow, I found 32 in queue!
Hey, I've just put 2048 to queue!
Wow, I found 256 in queue!
Wow, I found 4 in queue!
Wow, I found 2048 in queue!
Hey, I've just put 64 to queue!
Hey, I've just put 512 to queue!
Wow, I found 64 in queue!
Hey, I've just put 8 to queue!
Hey, I've just put 4096 to queue!
Wow, I found 512 in queue!
Wow, I found 8 in queue!
Wow, I found 4096 in queue!
Hey, I've just put 8192 to queue!
Hey, I've just put 524288 to queue!
Wow, I found 524288 in queue!
Hey, I've just put 65536 to queue!
Hey, I've just put 4194304 to queue!
Wow, I found 8192 in queue!
Wow, I found 65536 in queue!
Wow, I found 4194304 in queue!
Wow, I found 16384 in queue!
Hey